In [266]:
import pandas as pd
import bson
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import io, bson, multiprocessing
import tensorflow as tf
from sklearn import preprocessing
from sklearn.utils import shuffle

In [267]:
with open("train_example.bson",'rb')as train:
    df=pd.DataFrame(bson.decode_all(train.read()),index=None)
    #df.set_index('_id', inplace=True)
df[0:11]   

,_id,category_id,imgs
0,0,1000010653,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...
1,1,1000010653,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...
2,2,1000004079,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...
3,3,1000004141,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...
4,4,1000015539,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...
5,5,1000010653,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...
6,6,1000005744,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...
7,7,1000004079,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...
8,8,1000010667,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...
9,9,1000018290,[{'picture': b'\xff\xd8\xff\xe0\x00\x10JFIF\x0...


In [268]:
#get binary image data
df['imgs'] = df['imgs'].apply(lambda rec: rec[0]['picture'])

In [269]:
#convert to raw image
df['imgs'] = df['imgs'].apply(lambda img: Image.open(io.BytesIO(img)))

In [270]:
df.columns[1]

'category_id'

df = df.sort_values(df.columns[1])

In [271]:
cat_unique = df.category_id.unique()
print (cat_unique)

[1000010653 1000004079 1000004141 1000015539 1000005744 1000010667
 1000018290 1000018306 1000010961 1000015309 1000007361 1000018294
 1000014287 1000005910 1000001859 1000010647 1000011423 1000012989
 1000012558 1000010061 1000005990 1000012993 1000014396 1000003787
 1000010706 1000007138 1000003191 1000010645 1000003977 1000005796
 1000013922 1000010683 1000010641 1000010461 1000014053 1000004085]


In [272]:
len_data =  df.shape[0]
img_pixel_val = []
img_category = [] 
for i in range(len_data):
    img_pixel_val.append(np.asarray(df.imgs[i]))
    img_category.append(df.category_id)

In [273]:
img_pixel_val = np.array(img_pixel_val, dtype=float)
img_pixel_val_flat = img_pixel_val.flatten().reshape(len_data, 97200)
img_pixel_val_flat.shape

(82, 97200)

In [274]:
img_pixel_val_flat.shape

(82, 97200)

In [275]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 3 columns):
_id            82 non-null int64
category_id    82 non-null int64
imgs           82 non-null object
dtypes: int64(2), object(1)
memory usage: 2.0+ KB


In [276]:
train_labels = df['category_id'].values

In [277]:
train_labels = np.reshape(train_labels, (-1,1))
train_labels.shape

(82, 1)

In [278]:
enc = preprocessing.OneHotEncoder(sparse=False)

In [279]:
train_labels = train_labels.reshape(-1,1)
#print(cat_unique.shape)
train_labels = enc.fit_transform(train_labels)

In [280]:
train_labels.shape

(82, 36)

In [281]:
train_labels_class = train_labels.shape[1]
print(train_labels_class)

36


In [282]:
comb =  np.concatenate((img_pixel_val_flat, train_labels), axis = 1)
comb.shape

(82, 97236)

# selecting only row with label 18 / 19/4

In [283]:
total_data = img_pixel_val_flat.shape[0]
total_feature = img_pixel_val_flat.shape[1]
total_labels = train_labels.shape[1]
data_select = np.empty((0,total_feature))
data_labels = np.empty((0,total_labels))
#data_select = np.empty((0))
data_labels

array([], shape=(0, 36), dtype=float64)

In [284]:
chk = []
for i in range(total_data):
    chk.append(np.argmax(train_labels[i]))
    
np.unique(chk, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35], dtype=int64),
 array([ 1,  1,  1,  1,  4,  1,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1, 36,  4,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  1,  2,
         2,  1], dtype=int64))

In [285]:
chk = []
for i in range(total_data):
    chk.append(np.argmax(train_labels[i]))
np.unique(chk, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35], dtype=int64),
 array([ 1,  1,  1,  1,  4,  1,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1, 36,  4,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  1,  2,
         2,  1], dtype=int64))

In [286]:
#train_values = np.empty((0))
cnt = 0
for i in range(total_data):
    if (np.argmax(train_labels[i]) == 18):
        if cnt == 0:
            data = np.reshape(img_pixel_val_flat[i], (1, total_feature))
            label = np.reshape(train_labels[i], (1, total_labels))
            data_select = data
            data_labels = label
            cnt += 1
            print(data_select.shape)
        else:
            data = np.reshape(img_pixel_val_flat[i], (1, total_feature))
            label = np.reshape(train_labels[i], (1, total_labels))
            data_select = np.append(data_select, data, axis=0)
            data_labels = np.append(data_labels, label, axis=0)
            cnt +=1
            

print(data_select.shape) 
print(data_labels.shape)

(1, 97200)
(36, 97200)
(36, 36)


In [287]:
data_4select = np.empty((0,total_feature))
data_4labels = np.empty((0,total_labels))
cnt = 0
for i in range(total_data):
    if (np.argmax(train_labels[i]) == 4):
        if cnt == 0:
            data = np.reshape(img_pixel_val_flat[i], (1, total_feature))
            label = np.reshape(train_labels[i], (1, total_labels))
            data_4select = data
            data_4labels = label
            cnt += 1
            #print(data_select.shape)
        else:
            data = np.reshape(img_pixel_val_flat[i], (1, total_feature))
            label = np.reshape(train_labels[i], (1, total_labels))
            data_4select = np.append(data_4select, data, axis=0)
            data_4labels = np.append(data_4labels, label, axis=0)
            cnt +=1
            
            
print(data_4select.shape) 
print(data_4labels.shape)

(4, 97200)
(4, 36)


In [288]:
cnt = 0
for i in range(total_data):
    if (np.argmax(train_labels[i]) == 4):
            data = np.reshape(img_pixel_val_flat[i], (1, total_feature))
            label = np.reshape(train_labels[i], (1, total_labels))
            data_4select = np.append(data_4select, data, axis=0)
            data_4labels = np.append(data_4labels, label, axis=0)
            cnt +=1
            
print(data_4select.shape) 
print(data_4labels.shape)

(8, 97200)
(8, 36)


In [289]:
data_19select = np.empty((0,total_feature))
data_19labels = np.empty((0,total_labels))
cnt = 0
for i in range(total_data):
    if (np.argmax(train_labels[i]) == 19):
        if cnt == 0:
            data = np.reshape(img_pixel_val_flat[i], (1, total_feature))
            label = np.reshape(train_labels[i], (1, total_labels))
            data_19select = data
            data_19labels = label
            cnt += 1
            #print(data_select.shape)
        else:
            data = np.reshape(img_pixel_val_flat[i], (1, total_feature))
            label = np.reshape(train_labels[i], (1, total_labels))
            data_19select = np.append(data_19select, data, axis=0)
            data_19labels = np.append(data_19labels, label, axis=0)
            cnt +=1
            
            
print(data_19select.shape) 
print(data_19labels.shape)

cnt = 0
for i in range(total_data):
    if (np.argmax(train_labels[i]) == 19):
            data = np.reshape(img_pixel_val_flat[i], (1, total_feature))
            label = np.reshape(train_labels[i], (1, total_labels))
            data_19select = np.append(data_19select, data, axis=0)
            data_19labels = np.append(data_19labels, label, axis=0)
            cnt +=1
            
print(data_19select.shape) 
print(data_19labels.shape)

(4, 97200)
(4, 36)
(8, 97200)
(8, 36)


# SMOTE DATA PREPARATION

In [290]:
comb_data = np.concatenate((data_select, data_4select, data_19select), axis=0)
comb_labels = np.concatenate((data_labels, data_4labels, data_19labels), axis=0)
max_ind = np.empty((0,1))
cnt = 0
for i in range(comb_labels.shape[0]):
     if cnt == 0:
            data = np.argmax(comb_labels[i])
            data = np.reshape(data, (1, -1))
            max_ind = data
            #print (max_ind.shape)
            cnt += 1
     else:
            data = np.argmax(comb_labels[i])
            data = np.reshape(data, (1, -1))
            #print (data)
            #print ('max',max_ind)
            max_ind = np.append(max_ind, data, axis=0)
            cnt +=1
            

print(comb_data.shape)
print(max_ind.shape)

(52, 97200)
(52, 1)


In [291]:
from imblearn.over_sampling import SMOTE
final_inp, int_label = SMOTE().fit_sample(np.asarray(comb_data), np.asarray(max_ind))
from collections import Counter 
print(sorted(Counter(int_label).items()))
int_label

C:\Users\saish\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[(4, 36), (18, 36), (19, 36)]


array([18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
       18, 18,  4,  4,  4,  4,  4,  4,  4,  4, 19, 19, 19, 19, 19, 19, 19,
       19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
       19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4], dtype=int64)

In [292]:
enc = preprocessing.OneHotEncoder(sparse=False)
int_label = int_label.reshape(-1,1)
final_label = enc.fit_transform(int_label)
final_label.shape

(108, 3)

# shuffle data

In [293]:
final_inp, final_label = shuffle(final_inp, final_label)


# split data

In [294]:
total_rows = final_inp.shape[0]
train_split = int(total_rows * 0.8)
rem_rows = total_rows - train_split
print(rem_rows)
val_split = int(rem_rows * 0.5)
test_split = total_rows - (train_split + val_split)
print(train_split)
print(val_split)
print(test_split)
val_limit = train_split + val_split

22
86
11
11


In [295]:
train_img_pixel_val_flat = final_inp[0:train_split]
val_img_pixel_val_flat = final_inp[train_split:val_limit]
test_img_pixel_val_flat = final_inp[val_limit:total_rows]
row_len_train=train_img_pixel_val_flat.shape[0]
print(row_len_train)

86


In [297]:
train_train_labels = final_label[0:train_split]
val_train_labels = final_label[train_split:val_limit]
test_train_labels = final_label[val_limit:total_rows]

train_img_pixel_val_flat = final_inp[0:18]
print(train_img_pixel_val_flat.shape[0])
data = final_inp[36:54]
train_img_pixel_val_flat = np.append(train_img_pixel_val_flat, data, axis=0)
row_len_train=train_img_pixel_val_flat.shape[0]
print(row_len_train)

val_img_pixel_val_flat = final_inp[18:27]
print(val_img_pixel_val_flat.shape[0])
data = final_inp[54:63]
val_img_pixel_val_flat = np.append(val_img_pixel_val_flat, data, axis=0)
val_img_pixel_val_flat.shape[0]

test_img_pixel_val_flat = final_inp[27:36]
print(test_img_pixel_val_flat.shape[0])
data = final_inp[63:72]
test_img_pixel_val_flat = np.append(test_img_pixel_val_flat, data, axis=0)
test_img_pixel_val_flat.shape[0]

train_train_labels = final_label[0:18]
print((train_train_labels.shape))
label = final_label[36:54]
train_train_labels = np.append(train_train_labels, label, axis=0)
print((train_train_labels.shape))



val_train_labels = final_label[18:27]
print((val_train_labels.shape))
label = final_label[54:63]
val_train_labels = np.append(val_train_labels, label, axis=0)
print((val_train_labels.shape))
#print(val_train_labels)

test_train_labels = final_label[27:36]
print((test_train_labels.shape))
label = final_label[63:72]
test_train_labels = np.append(test_train_labels, label, axis=0)
print((test_train_labels.shape))

In [298]:
train_values = np.empty((0))
for i in range(len(train_train_labels)):
     train_values = np.append(train_values, np.argmax(train_train_labels[i]))
        
train_values

array([ 2.,  2.,  0.,  0.,  1.,  2.,  1.,  2.,  2.,  1.,  0.,  0.,  0.,
        1.,  1.,  0.,  0.,  1.,  0.,  2.,  0.,  1.,  1.,  2.,  1.,  2.,
        1.,  2.,  0.,  0.,  2.,  0.,  1.,  2.,  0.,  2.,  1.,  1.,  2.,
        1.,  0.,  1.,  1.,  2.,  0.,  2.,  1.,  2.,  1.,  2.,  1.,  0.,
        0.,  1.,  1.,  0.,  2.,  2.,  2.,  0.,  0.,  2.,  0.,  0.,  0.,
        0.,  2.,  2.,  1.,  2.,  1.,  0.,  2.,  1.,  2.,  2.,  1.,  0.,
        0.,  2.,  0.,  2.,  2.,  1.,  1.,  1.])

In [299]:
np.unique(train_values, return_counts=True)

(array([ 0.,  1.,  2.]), array([28, 28, 30], dtype=int64))

In [300]:
val_values = np.empty((0))
for i in range(len(val_train_labels)):
     val_values = np.append(val_values, np.argmax(val_train_labels[i]))
        
val_values

array([ 0.,  0.,  1.,  1.,  1.,  0.,  1.,  2.,  1.,  2.,  0.])

In [301]:
test_values = np.empty((0))
for i in range(len(test_train_labels)):
     test_values = np.append(test_values, np.argmax(test_train_labels[i]))
        
test_values

array([ 1.,  0.,  0.,  0.,  2.,  2.,  2.,  2.,  0.,  1.,  1.])

train_img_pixel_val_flat, train_train_labels =shuffle(train_img_pixel_val_flat, train_train_labels, random_state = 0 )
val_img_pixel_val_flat, val_train_labels =shuffle(val_img_pixel_val_flat, val_train_labels, random_state = 0 )
test_img_pixel_val_flat, test_train_labels =shuffle(test_img_pixel_val_flat, test_train_labels, random_state = 0 )

In [303]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import tempfile

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

FLAGS = None

def deepnn(x, n_class):
    
  num_rows, num_cols = x.get_shape().as_list()
  print('rows', num_rows)
    
  #rows, columns = map(lambda i: i.value, x.get_shape())
  #print('rows', rows)

  with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 180, 180, 3])

  with tf.name_scope('conv1'):
    W_conv1 = weight_variable([5, 5, 3, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

  with tf.name_scope('pool1'):
     h_pool1 = max_pool_2x2(h_conv1)
 
  with tf.name_scope('conv2'):
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
 
  with tf.name_scope('pool2'):
    h_pool2 = max_pool_2x2(h_conv2)

  with tf.name_scope('conv3'):
    W_conv3 = weight_variable([5, 5, 64, 128])
    b_conv3 = bias_variable([128])
    h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
 
  with tf.name_scope('pool3'):
    h_pool3 = max_pool_2x2(h_conv3)

  with tf.name_scope('fc1'):
    W_fc1 = weight_variable([23 * 23 * 128, 1024])
    b_fc1 = bias_variable([1024])

    h_pool3_flat = tf.reshape(h_pool3, [-1, 23*23*128])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)

  with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

  
  with tf.name_scope('fc2'):
    W_fc2 = weight_variable([1024, n_class])
    b_fc2 = bias_variable([n_class])

    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
  return y_conv, keep_prob

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def main():
    
    
    
    x  = tf.placeholder(tf.float32, [None, 97200])
    print('x', x.shape)
    y_ = tf.placeholder(tf.float32, [None, 3])
   
    
    

    y_conv, keep_prob = deepnn(x, 3)
    
    
    with tf.name_scope('loss'):
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_,
                                                            logits=y_conv)
    cross_entropy = tf.reduce_mean(cross_entropy)

    with tf.name_scope('adam_optimizer'):
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
        
    with tf.name_scope('prediction'):
        prediction = tf.argmax(y_conv, 1)
        
    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(prediction, tf.argmax(y_, 1))
        correct_prediction = tf.cast(correct_prediction, tf.float32)
        accuracy = tf.reduce_mean(correct_prediction)

    graph_location = tempfile.mkdtemp()
    print('Saving graph to: %s' % graph_location)
    train_writer = tf.summary.FileWriter(graph_location)
    train_writer.add_graph(tf.get_default_graph())

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(100):
            inp_size = np.arange(train_img_pixel_val_flat.shape[0])
            np.random.shuffle(inp_size)
            #print (inp_size)
            #
            shuffle_tr_img_pixel_val_flat = train_img_pixel_val_flat[inp_size]
            shuffle_tr_test_labels = train_train_labels[inp_size]
            #print(shuffle_tr_img_pixel_val_flat.shape)
            #print(shuffle_tr_test_labels.shape)
            #
            if i % 20 == 0:
                mode = 'train'
                
                train_accuracy = accuracy.eval(feed_dict={x: shuffle_tr_img_pixel_val_flat, y_: shuffle_tr_test_labels, keep_prob: 1.0})
                
                #arr = prediction.eval(feed_dict={x: img_pixel_val_flat, y_: b, keep_prob: 1.0})
                
                #print('arr',arr)
                print('step %d, training accuracy %g' % (i, train_accuracy))
            
            train_step.run(feed_dict={x: shuffle_tr_img_pixel_val_flat, y_: shuffle_tr_test_labels, keep_prob: 0.5})
        ##
        mode = 'val'
        print('Validation accuracy %g' % accuracy.eval(feed_dict={
        x: val_img_pixel_val_flat, y_: val_train_labels, keep_prob: 1.0}))
        
        ##
        mode = 'test'
        arr = prediction.eval(feed_dict={x: test_img_pixel_val_flat, y_: test_train_labels, keep_prob: 1.0})
                
        ##
        mode = 'val'
        print('test accuracy %g' % accuracy.eval(feed_dict={
        x: test_img_pixel_val_flat, y_: test_train_labels, keep_prob: 1.0}))
        
        print('arr',arr)

if __name__ == '__main__':
    main()


x (?, 97200)
rows None
Saving graph to: C:\Users\saish\AppData\Local\Temp\tmp3_7oz4o6
step 0, training accuracy 0.337209
step 20, training accuracy 0.988372
step 40, training accuracy 1
step 60, training accuracy 1
step 80, training accuracy 1
Validation accuracy 1
test accuracy 1
arr [1 0 0 0 2 2 2 2 0 1 1]
